In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## データの説明

心血管疾患(CVD)は、世界的に第1位の死因であり、毎年1,790万人の命を奪っており、全世界の死亡者数の31%を占めている。
このデータセットには、心不全による死亡率を予測するために使用できる12の特徴が含まれている。

心血管疾患に罹患している人、または心血管疾患のリスクが高い人（高血圧、糖尿病、高脂血症などの1つ以上の危険因子がある場合、または既に疾患が確立している場合）は、早期発見と管理が必要であり、そのためには機械学習モデルが大きな助けとなる。

そこで、このノートブックでは与えられたデータをtrainとtestに分けて心不全により死亡するか予測するモデルを構築することを目的とした。

### データの項目について

- age　患者の年齢
- anaemia　貧血の有無（赤血球またはヘモグロビンの減少）
- creatinine_phosphokinase　血中のCPK（クレアチニンホスホキナーゼ）酵素のレベル
- diabetes　 糖尿病の有無
- ejection_fraction　
- high_blood_pressure　高血圧の有無
- platelets 血液中の血小板数
- serum_creatinine　  血液中のクレアチニン濃度
- serum_sodium　血液中のナトリウムのレベル
- sex　性別
- smoking　喫煙の有無
- time　（詳しい説明の記載なし）
- DEATH_EVENT　死亡事象の有無

質的変数の意味は以下の通り
- Sex - Gender of patient Male = 1, Female =0
- Age - Age of patient
- Diabetes - 0 = No, 1 = Yes
- Anaemia - 0 = No, 1 = Yes
- High_blood_pressure - 0 = No, 1 = Yes
- Smoking - 0 = No, 1 = Yes
- DEATH_EVENT - 0 = No, 1 = Yes

## データの読み込み

In [ ]:
data=pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

## データの概観

In [ ]:
data.columns

In [ ]:
data.head()

anaemia,diabetes,high_blood_pressure,smoking,sexの項目は0か1の表記になっている。

In [ ]:
data.shape

In [ ]:
data.info()

欠損値がないようなのでデータを削る必要はない。

In [ ]:
data.describe()

## 各項目と死亡事象の関係を見る。


### 質的変数について

In [ ]:
data[['anaemia', 'DEATH_EVENT']].groupby(['anaemia'], as_index=False).mean()

In [ ]:
data[['smoking', 'DEATH_EVENT']].groupby(['smoking'], as_index=False).mean()

In [ ]:
data[['diabetes', 'DEATH_EVENT']].groupby(['diabetes'], as_index=False).mean()

In [ ]:
data[['high_blood_pressure', 'DEATH_EVENT']].groupby(['high_blood_pressure'], as_index=False).mean()

In [ ]:
data[['sex', 'DEATH_EVENT']].groupby(['sex'], as_index=False).mean()

diabetes(糖尿病)の項目については関係がなさそうなので、説明変数から除外する。

### 量的変数について

まず、timeの項目についての説明が不十分であるのでこれは説明変数として考慮するのは危険である。

### age(年齢)について

ヒストグラム

In [ ]:
fig = px.histogram(data, x="age", color="DEATH_EVENT", marginal="violin", hover_data=data.columns, 
                   title ="年齢の分布", 
                   labels={"age": "年齢" },
                   template="plotly_white",
                   color_discrete_map={"0": "RebeccaPurple", "1": "MediumPurple"}
                  )
fig.show()

In [ ]:
sns.boxplot(x = data.age, color = 'teal')
plt.show()

左室駆出率（分画）〈left ventricular ejection fraction；LVEF〉 包括的な収縮期の左室機能に関係する指標の1つで、左室駆出率（LVEF）が50～55％未満のときには左室収縮機能不全状態と推定されることが多い。

### CPK酵素の濃度について

In [ ]:
fig = px.histogram(data, x="creatinine_phosphokinase", color="DEATH_EVENT", marginal="violin", hover_data=data.columns, 
                   title ="CPKの分布", 
                   labels={"creatinine_phosphokinase": "CPK酵素のレベル" },
                   template="plotly_white",
                   color_discrete_map={"0": "RebeccaPurple", "1": "MediumPurple"}
                  )
fig.show()

In [ ]:
sns.boxplot(x = data.creatinine_phosphokinase, color = 'teal')
plt.show()

左室駆出率について

In [ ]:
fig = px.histogram(data, x="ejection_fraction", color="DEATH_EVENT", marginal="violin", hover_data=data.columns, 
                   title ="出血の割合", 
                   labels={"ejection_fraction": "出血の割合" },
                   template="plotly_white",
                   color_discrete_map={"0": "RebeccaPurple", "1": "MediumPurple"}
                  )
fig.show()

In [ ]:
sns.boxplot(x = data.ejection_fraction, color = 'teal')
plt.show()

### 血小板枚数について

In [ ]:
fig = px.histogram(data, x="platelets", color="DEATH_EVENT", marginal="violin", hover_data=data.columns, 
                   title ="血小板数の分布", 
                   labels={"platelets": "血液中の血小板数" },
                   template="plotly_white",
                   color_discrete_map={"0": "RebeccaPurple", "1": "MediumPurple"}
                  )
fig.show()

In [ ]:
sns.boxplot(x = data.platelets, color = 'teal')
plt.show()

### 血液中のクレアチニン濃度について

In [ ]:
fig = px.histogram(data, x="serum_creatinine", color="DEATH_EVENT", marginal="violin", hover_data=data.columns, 
                   title ="血液中のクレアチニン濃度の分布", 
                   labels={"serum_creatinine": "血液中のクレアチニン濃度" },
                   template="plotly_white",
                   color_discrete_map={"0": "RebeccaPurple", "1": "MediumPurple"}
                  )
fig.show()

In [ ]:
sns.boxplot(x = data.serum_creatinine, color = 'teal')
plt.show()

Mm血液中のナトリウムのレベル

In [ ]:
fig = px.histogram(data, x="serum_sodium", color="DEATH_EVENT", marginal="violin", hover_data=data.columns, 
                   title ="血液中のナトリウムのレベルの分布", 
                   labels={"serum_sodium": "血液中のナトリウムのレベル" },
                   template="plotly_white",
                   color_discrete_map={"0": "RebeccaPurple", "1": "MediumPurple"}
                  )
fig.show()

In [ ]:
sns.boxplot(x = data.serum_sodium, color = 'teal')
plt.show()

次に、相関表を作ってみる。

In [ ]:
data1=data[['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium', 'DEATH_EVENT']]
corr_mat = data1.corr(method='pearson')
corr_mat[['DEATH_EVENT']]

creatinine_phosphokinase,plateletsの項目については相関が小さいようである。

以上のことから、age,ejection_fraction,serum_creatinine,serum_sodiumの4項目を量的変数として説明変数に組み込む。

## 前処理

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.loc[ data['age'] <= 40, 'ageband'] = 0
data.loc[(data['age'] > 40) & (data['age'] <= 50), 'ageband'] = 1
data.loc[(data['age'] > 50) & (data['age'] <= 60), 'ageband'] = 2
data.loc[(data['age'] > 60) & (data['age'] <= 70), 'ageband'] = 3
data.loc[(data['age'] > 70) & (data['age'] <= 80), 'ageband'] = 4
data.loc[(data['age'] > 80) & (data['age'] <= 90), 'ageband'] = 5
data.loc[ data['age'] > 90, 'ageband']=6
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2,random_state=10)

In [ ]:
x_train=train[['smoking', 'ageband','ejection_fraction','serum_creatinine','serum_sodium','anaemia','diabetes','high_blood_pressure','sex']]
y_train=train[['DEATH_EVENT']]

In [ ]:
x_test=test[['smoking','ageband','ejection_fraction','serum_creatinine','serum_sodium','anaemia','diabetes','high_blood_pressure','sex']]
y_test=test[['DEATH_EVENT']]

## モデリング

ロジスティック回帰を考えてみる

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train,y_train)


### アンサンブル平均を考える

ランダムフォレスト

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
rfcl = RandomForestClassifier(criterion = 'entropy', class_weight={0:.5,1:.5}, max_depth = 5, min_samples_leaf=5)

In [ ]:
rfcl.fit(x_train,y_train)
rfcl_score=rfcl.score(x_test , y_test)
y_pred_lr=lr.predict(x_test)
print("ロジスティック回帰のaccuracyは{0:.4f}".format(rfcl_score))
print('-'*50)
print("混同行列は以下の通り")
print(metrics.confusion_matrix(y_test,y_pred_lr))

In [ ]:
y_pred_rfcl = rfcl.predict(x_test)
rfcl_score=rfcl.score(x_test , y_test)
print("ランダムフォレストのaccuracyは{0:.4f}".format(rfcl_score))
print('-'*50)
print("混同行列は以下の通り")
print(metrics.confusion_matrix(y_test,y_pred_rfcl))

Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abcl = AdaBoostClassifier( n_estimators= 20)
abcl = abcl.fit(x_train, y_train)

y_pred_abcl = abcl.predict(x_test)
abcl_score=abcl.score(x_test, y_test)
print("アダブーストのaccuracyは{0:.4f}".format(abcl_score))
print('-'*50)
print("混同行列は以下の通り")
print(metrics.confusion_matrix(y_test,y_pred_abcl))

Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

bgcl = BaggingClassifier(n_estimators=10, max_samples= .7, bootstrap=True)
bgcl = bgcl.fit(x_train, y_train)
y_pred_bgcl = bgcl.predict(x_test)
bgcl_score=bgcl.score(x_test, y_test)

print("バギングのaccuracyは{0:.4f}".format(bgcl_score))
print('-'*50)
print("混同行列は以下の通り")
print(metrics.confusion_matrix(y_test,y_pred_bgcl))

勾配ブースティング

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbcl = GradientBoostingClassifier(n_estimators = 50, learning_rate = 0.05)
gbcl = gbcl.fit(x_train, y_train)
y_pred_gbcl = gbcl.predict(x_test)
gbcl_score=gbcl.score(x_test , y_test)

print("勾配ブースティングのaccuracyは{0:.4f}".format(gbcl_score))
print('-'*50)
print("混同行列は以下の通り")
print(metrics.confusion_matrix(y_test,y_pred_gbcl))

以上からモデルを比較すると、ランダムフォレストが一番良いモデルとなっている。

試しに、このモデルのうち三つ(ランダムフォレスト、ロジスティック回帰、アダブースト)を融合することを考えてみる

In [ ]:
print("ロジスティック回帰のaccuracyは{0:.4f}",metrics.accuracy_score(y_test, y_pred_lr))
print("ランダムフォレストのaccuracyは{0:.4f}",metrics.accuracy_score(y_test, y_pred_rfcl))
print("アダブーストのaccuracyは{0:.4f}",metrics.accuracy_score(y_test, y_pred_abcl))
print("バギングのaccuracyは{0:.4f}",metrics.accuracy_score(y_test, y_pred_bgcl))
print("勾配ブースティングのaccuracyは{0:.4f}",metrics.accuracy_score(y_test, y_pred_gbcl))

勾配ブースティングが一番良いモデルである。

In [ ]:
'''
y_pred=[]
for i in range (0,60):
    a=y_pred_lr[i]*0+y_pred_rfcl[i]*0.2+y_pred_abcl[i]*0.1+y_pred_bgcl[i]*0.1+y_pred_gbcl[i]*0.6
    if a>=0.7: y_pred.append(1)
    else :y_pred.append(0)
y_pred
'''

## 結果の解釈

まず、質的変数についてだが途中に書いた通り糖尿病の項目は意外にもあまり心不全による死亡には影響しないことがわかる。喫煙については喫煙した方が心不全で死亡する可能性が低いという結果が出ているがこれは一般的に考えて不可解な部分であるので今回用いたデータ特有の特徴なのではないかと考える。(しかし、相関が大きいので今回のモデルには組み込むことにした。)

次に量的変数についてだが、ヒストグラム、相関表をからもわかるように年齢や血液中のクレアチニン濃度が高いほど死亡率が高いことがわかる。それに対し、左室駆出率が低い方が死亡率が高くなることもわかる。反対に、CPK酵素のレベル,血小板数はあまり関係ないことがわかる。

In [ ]:
c=0
for i in range(0,data.shape[0]):
    if data.loc[i,'DEATH_EVENT']==1: c+=1
c

### モデルについて
今回のモデルではどんなに高くても約79パーセントの正確さにとどまった。このモデルは、実際には病院で患者に適用されることを想定して作るモデルであるのでこの正確さでは低すぎるであろう。このような結果になってしまった要因は、もちろんモデルが最適解ではなかったことも考えられるが、一番はデータセットの少なさにあると思う。trainデータとtestデータの両方を合わせて約300個はあまりにも少なすぎる。さらに、上の計算からわかるように死亡事象に関するデータは96個しかないのでモデルの正確さの低さは以下仕方ないのではと考える。新たなデータが得られたこのモデルはさらに改善して行くと思う。

## 今後の展望

上記のように、データが少なすぎるのでデータを収集することが最優先である。特に、喫煙の有無と死亡率の関係には負の相関が生じてしまっているのでここのデータが非常に重要である。また、今回は箱ひげ図からもわかるように外れ値がいくつかあったがこの外れ値が異常値であるかどうかはわからなかったので削除せず残したので、ここにモデルの正確さが改善される余地があると思う。